In [2]:
import os
import pandas as pd
import numpy as np
from nltk.tokenize import TweetTokenizer
from collections import Counter

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Bidirectional, Dropout, Activation, Conv1D, GlobalMaxPooling1D
from keras import regularizers, initializers


Using TensorFlow backend.


In [145]:
def create_dictionary(texts, vocab_size):
    """
    Creates a dictionary that maps words to ids. More frequent words have lower ids.
    The dictionary contains at the vocab_size-1 most frequent words (and a placeholder '<unk>' for unknown words).
    The place holder has the id 0.
    """
    counter = Counter()
    for tokens in texts:
        counter.update(tokens)
    vocab = [w for w, c in counter.most_common(vocab_size - 1)]
    word_to_id = {w: (i + 1) for i, w in enumerate(vocab)}
    word_to_id[UNKNOWN_TOKEN] = 0
    return word_to_id


def to_ids(words, dictionary):
    """
    Takes a list of words and converts them to ids using the word2id dictionary.
    """
    ids = []
    for word in words:
        ids.append(dictionary.get(word, dictionary[UNKNOWN_TOKEN]))
    return ids


def read_data(train_file, dev_file):
    tokenizer = TweetTokenizer()
    trainDF = pd.read_csv(train_file, sep='\t')
    devDF = pd.read_csv(dev_file, sep='\t')

    allDF = pd.concat([trainDF, devDF], ignore_index=True)
    allDF = allDF.reindex(np.random.permutation(allDF.index))
    allDF.insert(1, 'tweet_tokenized', (allDF['Tweet'].apply(lambda x: tokenizer.tokenize(x))))

    word2id = create_dictionary(allDF["tweet_tokenized"], VOCAB_SIZE)

    allDF.insert(1, 'tweet_ids', (allDF['Tweet'].apply(lambda x: to_ids(x, dictionary=word2id))))

    allDF['all'] = allDF.iloc[:, -11:].values.tolist()
    total = len(allDF)
    trainend = int(total * 0.8)
    devend = trainend + int(total * 0.1)
    return allDF.iloc[:trainend, :], allDF.iloc[trainend:devend, :], allDF.iloc[devend:, :]


def evaluate(predictions, y_test):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    all_correct = 0
    for i, pred in enumerate(predictions):
        for j, em in enumerate(pred):
            if em >= 0.5:
                if y_test[i][j] == 1:
                    tp += 1
                else:
                    fp += 1
            if em <= 0.5:
                if y_test[i][j] == 1:
                    fn += 1
                else:
                    tn += 1
            if tp + tn == 11:
                all_correct += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2*(precision * recall) / (precision + recall)

    print("F1: {}\nPrecision: {}\nRecall: {}\nCompletely correct: {}".format(f1, precision, recall, all_correct))


In [122]:
data_dir = 'D:/3_Programming/1_Studium/Python/SemEval2018_Task1_5/data/'
train_file = os.path.join(data_dir, '2018-E-c-En-train.txt')
dev_file = os.path.join(data_dir, '2018-E-c-En-dev.txt')

VOCAB_SIZE = 100000
MAX_LEN = 100
BATCH_SIZE = 32
EMBEDDING_SIZE = 100
HIDDEN_SIZE = 50
EPOCHS = 10  # Standard 10
UNKNOWN_TOKEN = "<unk>"
EMOTIONS = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love',
            'optimism', 'pessimism', 'sadness', 'surprise', 'trust']

In [123]:
trainDF, devDF, testDF = read_data(train_file, dev_file)
x_train = sequence.pad_sequences(np.array(trainDF['tweet_ids']), maxlen=MAX_LEN)
x_dev = sequence.pad_sequences(np.array(devDF['tweet_ids']), maxlen=MAX_LEN)
x_test = sequence.pad_sequences(np.array(testDF['tweet_ids']), maxlen=MAX_LEN)
y_train = np.array([trainDF['all']])[0]
y_dev = np.array([devDF['all']])[0]
y_test = np.array([testDF['all']])[0]

In [136]:
cnn_model = Sequential()
cnn_model.add(Embedding(VOCAB_SIZE, EMBEDDING_SIZE))
cnn_model.add(Conv1D(2 * HIDDEN_SIZE,
                     kernel_size=3,
                     activation='tanh',
                     strides=1,
                     padding='valid',
                     ))
# cnn_model.add(GlobalMaxPooling1D())
# cnn_model.add(Dropout(0.5))
# cnn_model.add(Conv1D(2 * HIDDEN_SIZE,
#                      kernel_size=3,
#                      activation='tanh',
#                      strides=1,
#                      padding='valid',
#                      ))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(HIDDEN_SIZE, activation='tanh'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(11, activation='sigmoid')) #11 = no of classes

In [137]:
from keras.optimizers import SGD, adam

opt = adam(lr=0.01)
cnn_model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])


In [138]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopper = EarlyStopping(monitor='val_acc', patience=5, mode='max')
checkpoint = ModelCheckpoint(data_dir+'model.m', save_best_only=True, monitor='val_acc', mode='max')


In [135]:
#Todo:Functional API class:weights 0,1 11 outputs


In [149]:
cnn_model.fit(
    x_train,
    y_train,
    batch_size=BATCH_SIZE,
    # callbacks=[early_stopper, checkpoint],
    epochs=EPOCHS,
    validation_data=(x_dev, y_dev),
    class_weight={0: 1, 1: 2, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1},
    verbose=1
)


Train on 6179 samples, validate on 772 samples
Epoch 1/10
  32/6179 [..............................] - ETA: 27s - loss: 0.3891 - acc: 0.8352

  64/6179 [..............................] - ETA: 28s - loss: 0.4514 - acc: 0.8168

  96/6179 [..............................] - ETA: 28s - loss: 0.4847 - acc: 0.8021

 128/6179 [..............................] - ETA: 29s - loss: 0.4766 - acc: 0.8068

 160/6179 [..............................] - ETA: 29s - loss: 0.4953 - acc: 0.7983

...

6080/6179 [============================>.] - ETA: 0s - loss: 0.5386 - acc: 0.7856

6112/6179 [============================>.] - ETA: 0s - loss: 0.5383 - acc: 0.7855

6144/6179 [============================>.] - ETA: 0s - loss: 0.5382 - acc: 0.7855

6176/6179 [============================>.] - ETA: 0s - loss: 0.5377 - acc: 0.7856

6179/6179 [==============================] - 28s 5ms/step - loss: 0.5376 - acc: 0.7857 - val_loss: 0.4615 - val_acc: 0.7856


In [141]:
from keras.models import load_model
best_model = load_model(data_dir+'model.m')

In [142]:
predictions = best_model.predict(x_test)
print(y_test[0])
print(predictions[0])

[0 0 0 0 0 0 0 0 0 0 0]
[ 0.05820267  0.07735149  0.05499275  0.02505347  0.87888688  0.43989077
  0.54356372  0.03238969  0.22662053  0.04491712  0.03687851]


In [146]:
evaluate(predictions,y_test)

F1: 0.27632915678970543
Precision: 0.3633125556544969
Recall: 0.22295081967213115
Completely correct: 2
